# 🧬 Módulo 2: Representación y Visualización Molecular
## Actividad 2.2: Notaciones Químicas

<div align="center">
  
**Universidad de Caldas - Departamento de Química**  
*Introducción a la Química Computacional (173G7G)*  
**Profesor:** José Mauricio Rodas Rodríguez

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/maurorodas/Quimica_computacional_173G7G/blob/main/modulo_02_representacion_molecular/02_notaciones_quimicas.ipynb)

</div>

---

## 🎯 Objetivos de Aprendizaje

Al finalizar esta actividad, serás capaz de:
- Comprender las notaciones SMILES, InChI y SMARTS
- Convertir entre diferentes notaciones químicas
- Generar estructuras 2D y 3D desde notaciones textuales
- Utilizar SMARTS para búsquedas de subestructuras
- Validar y canonizar notaciones químicas
- Aplicar notaciones en química computacional práctica

---

## 📚 Contenido

1. [Introducción a las Notaciones Químicas](#1-introduccion)
2. [SMILES: Simplified Molecular Input Line Entry System](#2-smiles)
3. [InChI: International Chemical Identifier](#3-inchi)
4. [SMARTS: SMiles ARbitrary Target Specification](#4-smarts)
5. [Conversión entre Notaciones](#5-conversiones)
6. [Búsqueda de Subestructuras](#6-subestructuras)
7. [Ejercicios Prácticos](#7-ejercicios)

---

---

## 📦 Instalación e Importación

In [ ]:
# Instalación en Google Colab
import sys
if 'google.colab' in sys.modules:
    !pip install rdkit -q
    !pip install pillow -q
    !pip install matplotlib -q
    print("✓ Librerías instaladas")

In [ ]:
# Importar librerías
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Draw, inchi
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# Configuración para visualización
IPythonConsole.ipython_useSVG = True

print(f"✓ RDKit versión: {Chem.rdBase.rdkitVersion}")
print("✓ Bibliotecas importadas correctamente")

---

## 1. Introducción a las Notaciones Químicas <a id="1-introduccion"></a>

### ¿Por qué usar notaciones químicas?

Las notaciones químicas permiten representar moléculas como **cadenas de texto**, lo que facilita:

- 📝 **Almacenamiento:** Bases de datos con millones de compuestos
- 🔍 **Búsqueda:** Encontrar compuestos específicos o con características particulares
- 📤 **Comunicación:** Compartir estructuras de forma compacta
- 🖥️ **Programación:** Integrar química en código

### Principales notaciones

| Notación | Propósito | Ejemplo |
|----------|-----------|----------|
| **SMILES** | Representación simple y legible | `CCO` (etanol) |
| **InChI** | Identificador único estándar | `InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3` |
| **SMARTS** | Patrones de búsqueda | `[OH]` (alcoholes) |

---

## 2. SMILES: Simplified Molecular Input Line Entry System <a id="2-smiles"></a>

SMILES es la notación más popular y legible para representar moléculas.

### 📖 Reglas básicas de SMILES

#### Átomos
- **Carbono:** `C` (implícito con hidrógenos)
- **Otros átomos:** Símbolo entre corchetes: `[N]`, `[O]`, `[Cl]`
- **Aromáticos:** Minúsculas: `c` (benceno aromático)

#### Enlaces
- **Simple:** `-` (opcional, puede omitirse)
- **Doble:** `=`
- **Triple:** `#`
- **Aromático:** `:` (generalmente implícito)

#### Ramificaciones
- Usar paréntesis: `CC(C)C` (isobutano)

#### Ciclos
- Números para cerrar ciclos: `C1CCCCC1` (ciclohexano)

### 🧪 Ejemplos básicos

In [ ]:
# Ejemplos de moléculas simples con SMILES

moleculas = {
    "Metano": "C",
    "Etano": "CC",
    "Propano": "CCC",
    "Etanol": "CCO",
    "Acetona": "CC(=O)C",
    "Ácido acético": "CC(=O)O",
    "Benceno": "c1ccccc1",
    "Tolueno": "Cc1ccccc1",
    "Fenol": "Oc1ccccc1",
    "Anilina": "Nc1ccccc1"
}

# Crear moléculas y mostrar
mols = [Chem.MolFromSmiles(smiles) for smiles in moleculas.values()]
img = Draw.MolsToGridImage(mols, molsPerRow=5, subImgSize=(200,200),
                           legends=list(moleculas.keys()))
display(img)

### 🔬 Moléculas con características especiales

In [ ]:
# Moléculas con dobles enlaces, ciclos y ramificaciones

moleculas_especiales = {
    "Eteno (etileno)": "C=C",
    "Etino (acetileno)": "C#C",
    "Isobutano": "CC(C)C",
    "Ciclohexano": "C1CCCCC1",
    "Ciclopenteno": "C1CC=CC1",
    "Ácido benzoico": "O=C(O)c1ccccc1",
    "Aspirina": "CC(=O)Oc1ccccc1C(=O)O",
    "Cafeína": "CN1C=NC2=C1C(=O)N(C(=O)N2C)C"
}

mols = [Chem.MolFromSmiles(smiles) for smiles in moleculas_especiales.values()]
img = Draw.MolsToGridImage(mols, molsPerRow=4, subImgSize=(250,250),
                           legends=list(moleculas_especiales.keys()))
display(img)

### 🎯 SMILES Canónico

Una misma molécula puede tener múltiples representaciones SMILES. El **SMILES canónico** es una representación única y estándar.

In [ ]:
# Diferentes SMILES para la misma molécula (etanol)

smiles_variantes = [
    "CCO",      # Canónico
    "OCC",      # Empezando por el oxígeno
    "C(O)C",    # Con paréntesis
]

print("🔍 Diferentes SMILES para etanol:\n")
for i, smi in enumerate(smiles_variantes, 1):
    mol = Chem.MolFromSmiles(smi)
    canonical = Chem.MolToSmiles(mol)
    print(f"{i}. SMILES original:  {smi}")
    print(f"   SMILES canónico:  {canonical}")
    print()

# Visualizar que todas son la misma molécula
mols = [Chem.MolFromSmiles(smi) for smi in smiles_variantes]
img = Draw.MolsToGridImage(mols, molsPerRow=3, subImgSize=(200,200),
                           legends=[f"SMILES: {smi}" for smi in smiles_variantes])
display(img)

### 💡 Ejercicio: Escribir SMILES

In [ ]:
# Escribe el SMILES de las siguientes moléculas:
# 1. Butano (4 carbonos en cadena)
# 2. Propanona (acetona con 3 carbonos)
# 3. Ácido propanoico
# 4. Ciclopropano (3 carbonos en ciclo)

mi_butano = ""  # TU CÓDIGO AQUÍ
mi_propanona = ""  # TU CÓDIGO AQUÍ
mi_acido = ""  # TU CÓDIGO AQUÍ
mi_ciclopropano = ""  # TU CÓDIGO AQUÍ

# Verificar tus respuestas
mis_smiles = [mi_butano, mi_propanona, mi_acido, mi_ciclopropano]
nombres = ["Butano", "Propanona", "Ácido propanoico", "Ciclopropano"]

mols = []
for nombre, smi in zip(nombres, mis_smiles):
    if smi:
        mol = Chem.MolFromSmiles(smi)
        if mol:
            mols.append(mol)
            print(f"✅ {nombre}: {Chem.MolToSmiles(mol)}")
        else:
            print(f"❌ {nombre}: SMILES inválido")
    else:
        print(f"⏸️ {nombre}: No completado")

if mols:
    img = Draw.MolsToGridImage(mols, molsPerRow=4, subImgSize=(200,200),
                               legends=nombres[:len(mols)])
    display(img)

---

## 3. InChI: International Chemical Identifier <a id="3-inchi"></a>

InChI es un identificador químico estándar internacional, diseñado para ser único y reproducible.

### 🎯 Características de InChI

- ✅ **Único:** Una sola representación por estructura
- ✅ **Estándar:** Desarrollado por IUPAC
- ✅ **Jerárquico:** Capas de información (fórmula, conectividad, estereoquímica)
- ✅ **InChIKey:** Versión hash de 27 caracteres para búsquedas

### 📖 Estructura de InChI

```
InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3
       │ │ └─ Capas de información
       │ └── Versión
       └──── Prefijo
```

### Capas de información:
- **Fórmula:** `C2H6O`
- **Conectividad:** `c1-2-3` (cómo están conectados los átomos)
- **Hidrógenos:** `h3H,2H2,1H3` (dónde están los H)

### 🔬 Generar InChI desde SMILES

In [ ]:
# Convertir SMILES a InChI

moleculas = [
    ("Etanol", "CCO"),
    ("Ácido acético", "CC(=O)O"),
    ("Benceno", "c1ccccc1"),
    ("Cafeína", "CN1C=NC2=C1C(=O)N(C(=O)N2C)C")
]

print("🔍 Conversión SMILES → InChI\n")
print("=" * 80)

for nombre, smiles in moleculas:
    mol = Chem.MolFromSmiles(smiles)
    inchi_str = Chem.MolToInchi(mol)
    inchi_key = Chem.MolToInchiKey(mol)
    
    print(f"\n📌 {nombre}")
    print(f"   SMILES:   {smiles}")
    print(f"   InChI:    {inchi_str}")
    print(f"   InChIKey: {inchi_key}")

### 🔄 Convertir InChI a molécula

In [ ]:
# Desde InChI podemos reconstruir la molécula

inchi_examples = {
    "Aspirina": "InChI=1S/C9H8O4/c1-6(10)13-8-5-3-2-4-7(8)9(11)12/h2-5H,1H3,(H,11,12)",
    "Ibuprofeno": "InChI=1S/C13H18O2/c1-9(2)8-11-4-6-12(7-5-11)10(3)13(14)15/h4-7,9-10H,8H2,1-3H3,(H,14,15)",
    "Paracetamol": "InChI=1S/C8H9NO2/c1-6(10)9-7-2-4-8(11)5-3-7/h2-5,11H,1H3,(H,9,10)"
}

mols = []
labels = []

for nombre, inchi_str in inchi_examples.items():
    mol = Chem.MolFromInchi(inchi_str)
    if mol:
        mols.append(mol)
        labels.append(nombre)
        print(f"✅ {nombre} cargado correctamente")

# Visualizar
img = Draw.MolsToGridImage(mols, molsPerRow=3, subImgSize=(300,300),
                           legends=labels)
display(img)

### 🆚 Comparación SMILES vs InChI

In [ ]:
# Comparar las dos notaciones

molecula = "Glucosa"
smiles = "C(C1C(C(C(C(O1)O)O)O)O)O"

mol = Chem.MolFromSmiles(smiles)
inchi_str = Chem.MolToInchi(mol)
inchi_key = Chem.MolToInchiKey(mol)

print(f"📊 Comparación de notaciones para {molecula}\n")
print("=" * 80)
print(f"\n📝 SMILES:")
print(f"   {smiles}")
print(f"   Longitud: {len(smiles)} caracteres")
print(f"\n🔬 InChI:")
print(f"   {inchi_str}")
print(f"   Longitud: {len(inchi_str)} caracteres")
print(f"\n🔑 InChIKey:")
print(f"   {inchi_key}")
print(f"   Longitud: {len(inchi_key)} caracteres (hash)")

print("\n" + "=" * 80)
print("\n💡 Observaciones:")
print("   • SMILES es más compacto y legible")
print("   • InChI es más robusto y único")
print("   • InChIKey es perfecto para búsquedas en bases de datos")

display(mol)

---

## 4. SMARTS: SMiles ARbitrary Target Specification <a id="4-smarts"></a>

SMARTS es un lenguaje de patrones para **buscar subestructuras** en moléculas.

### 🎯 Usos de SMARTS

- 🔍 Identificar grupos funcionales
- 🧪 Filtrar bibliotecas de compuestos
- 📊 Análisis de estructura-actividad
- 🔬 Validación de estructuras

### 📖 Sintaxis básica

| Patrón | Significado | Ejemplo |
|--------|-------------|----------|
| `[OH]` | Grupo hidroxilo | Alcoholes |
| `C(=O)O` | Ácido carboxílico | Ácidos |
| `[NX3]` | Nitrógeno con 3 conexiones | Aminas |
| `c1ccccc1` | Anillo aromático de 6 | Benceno |
| `[#6]` | Cualquier carbono | - |
| `[CX4]` | Carbono sp³ | - |

### 🔬 Ejemplos de patrones SMARTS

In [ ]:
# Definir patrones SMARTS para grupos funcionales

patrones = {
    "Alcohol": "[OH]",
    "Aldehído": "[CH]=O",
    "Cetona": "[C](=O)[C]",
    "Ácido carboxílico": "C(=O)[OH]",
    "Éster": "C(=O)O[C]",
    "Amina primaria": "[NX3;H2]",
    "Amida": "C(=O)N",
    "Anillo aromático": "c1ccccc1"
}

# Moléculas de prueba
moleculas_prueba = {
    "Etanol": "CCO",
    "Acetaldehído": "CC=O",
    "Acetona": "CC(=O)C",
    "Ác. acético": "CC(=O)O",
    "Acetato de etilo": "CC(=O)OCC",
    "Etilamina": "CCN",
    "Acetamida": "CC(=O)N",
    "Benceno": "c1ccccc1"
}

print("🔍 Detección de grupos funcionales con SMARTS\n")
print("=" * 100)

# Crear tabla de resultados
for nombre_mol, smiles in moleculas_prueba.items():
    mol = Chem.MolFromSmiles(smiles)
    print(f"\n📌 {nombre_mol} ({smiles})")
    
    grupos_encontrados = []
    for nombre_patron, patron in patrones.items():
        pattern = Chem.MolFromSmarts(patron)
        if mol.HasSubstructMatch(pattern):
            grupos_encontrados.append(nombre_patron)
    
    if grupos_encontrados:
        print(f"   ✅ Contiene: {', '.join(grupos_encontrados)}")
    else:
        print(f"   ❌ No contiene ningún grupo funcional definido")

### 🎨 Visualizar coincidencias SMARTS

In [ ]:
# Visualizar las coincidencias de un patrón SMARTS en una molécula

# Molécula: Aspirina
smiles_aspirina = "CC(=O)Oc1ccccc1C(=O)O"
mol_aspirina = Chem.MolFromSmiles(smiles_aspirina)

# Patrones a buscar
patrones_buscar = {
    "Éster": "C(=O)O[C]",
    "Ácido carboxílico": "C(=O)[OH]",
    "Anillo aromático": "c1ccccc1"
}

print("🎨 Visualización de patrones SMARTS en Aspirina\n")

for nombre, patron in patrones_buscar.items():
    pattern = Chem.MolFromSmarts(patron)
    matches = mol_aspirina.GetSubstructMatches(pattern)
    
    if matches:
        print(f"\n✅ {nombre} (patrón: {patron})")
        print(f"   Encontradas {len(matches)} coincidencia(s)")
        
        # Visualizar con átomos resaltados
        img = Draw.MolToImage(mol_aspirina, size=(400, 400), 
                             highlightAtoms=matches[0])
        display(img)
    else:
        print(f"\n❌ {nombre}: No encontrado")

---

## 5. Conversión entre Notaciones <a id="5-conversiones"></a>

### 🔄 Tabla de conversiones

```
    SMILES ←→ Molécula (RDKit Mol) ←→ InChI
      ↓                                  ↓
  SMARTS                            InChIKey
```

In [ ]:
# Funciones de conversión completas

def convertir_notaciones(input_str, tipo_entrada):
    """
    Convierte entre diferentes notaciones químicas.
    
    Args:
        input_str (str): String de entrada
        tipo_entrada (str): 'smiles' o 'inchi'
    
    Returns:
        dict: Diccionario con todas las notaciones
    """
    
    # Crear molécula
    if tipo_entrada.lower() == 'smiles':
        mol = Chem.MolFromSmiles(input_str)
    elif tipo_entrada.lower() == 'inchi':
        mol = Chem.MolFromInchi(input_str)
    else:
        return None
    
    if mol is None:
        return None
    
    # Generar todas las notaciones
    resultado = {
        'SMILES': Chem.MolToSmiles(mol),
        'SMILES_canónico': Chem.MolToSmiles(mol, canonical=True),
        'InChI': Chem.MolToInchi(mol),
        'InChIKey': Chem.MolToInchiKey(mol),
        'Fórmula': Chem.rdMolDescriptors.CalcMolFormula(mol),
        'Peso_molecular': Descriptors.MolWt(mol)
    }
    
    return resultado, mol

# Ejemplo de uso
print("🔄 Conversión completa de notaciones\n")
print("=" * 80)

# Desde SMILES
smiles_ejemplo = "CC(=O)Oc1ccccc1C(=O)O"  # Aspirina
print(f"\n📥 Entrada: SMILES = {smiles_ejemplo}")
print("\n📤 Salidas:")

resultado, mol = convertir_notaciones(smiles_ejemplo, 'smiles')
for clave, valor in resultado.items():
    print(f"   • {clave:20s}: {valor}")

print("\n" + "=" * 80)
display(mol)

---

## 6. Búsqueda de Subestructuras <a id="6-subestructuras"></a>

Una aplicación práctica muy importante es la **búsqueda de subestructuras** en bibliotecas de compuestos.

In [ ]:
# Crear una mini biblioteca de compuestos

biblioteca = {
    "Aspirina": "CC(=O)Oc1ccccc1C(=O)O",
    "Ibuprofeno": "CC(C)Cc1ccc(cc1)C(C)C(=O)O",
    "Paracetamol": "CC(=O)Nc1ccc(cc1)O",
    "Cafeína": "CN1C=NC2=C1C(=O)N(C(=O)N2C)C",
    "Morfina": "CN1CC[C@]23[C@@H]4[C@H]1CC5=C2C(=C(C=C5)O)O[C@H]3[C@H](C=C4)O",
    "Dopamina": "NCCc1ccc(O)c(O)c1",
    "Serotonina": "NCCc1c[nH]c2ccc(O)cc12",
    "Adrenalina": "CNC[C@H](O)c1ccc(O)c(O)c1",
    "Vitamina C": "O=C1OC(C(O)=C1O)C(O)C(O)CO",
    "Penicilina G": "CC1(C)SC2C(NC(=O)Cc3ccccc3)C(=O)N2C1C(=O)O"
}

print(f"📚 Biblioteca creada con {len(biblioteca)} compuestos")
print("\n" + "=" * 80)

In [ ]:
# Buscar compuestos que contengan un grupo funcional específico

def buscar_subestructura(biblioteca, patron_smarts, nombre_patron):
    """
    Busca compuestos que contengan una subestructura específica.
    """
    pattern = Chem.MolFromSmarts(patron_smarts)
    coincidencias = []
    
    for nombre, smiles in biblioteca.items():
        mol = Chem.MolFromSmiles(smiles)
        if mol and mol.HasSubstructMatch(pattern):
            coincidencias.append((nombre, smiles, mol))
    
    return coincidencias

# Buscar compuestos con diferentes grupos funcionales
busquedas = {
    "Grupos hidroxilo fenólicos": "c[OH]",
    "Ácidos carboxílicos": "C(=O)[OH]",
    "Aminas": "[NX3]",
    "Anillos aromáticos": "c1ccccc1"
}

for nombre_patron, patron in busquedas.items():
    print(f"\n🔍 Buscando: {nombre_patron} (SMARTS: {patron})")
    print("=" * 80)
    
    resultados = buscar_subestructura(biblioteca, patron, nombre_patron)
    
    if resultados:
        print(f"✅ Encontrados {len(resultados)} compuesto(s):\n")
        for nombre, smiles, mol in resultados:
            print(f"   • {nombre}")
        
        # Visualizar resultados
        mols = [r[2] for r in resultados]
        nombres = [r[0] for r in resultados]
        img = Draw.MolsToGridImage(mols, molsPerRow=3, subImgSize=(250,250),
                                   legends=nombres)
        display(img)
    else:
        print("❌ No se encontraron coincidencias")

---

## 7. Ejercicios Prácticos <a id="7-ejercicios"></a>

### 📝 Ejercicio 1: Conversión de notaciones

Convierte las siguientes moléculas entre SMILES e InChI:

In [ ]:
# Ejercicio 1: Completa el código

# a) Convierte estos SMILES a InChI
moleculas_ejercicio = {
    "Tolueno": "Cc1ccccc1",
    "Glicina": "NCC(=O)O",
    "Colesterol": "CC(C)CCCC(C)C1CCC2C1(CCC3C2CC=C4C3(CCC(C4)O)C)C"
}

# TU CÓDIGO AQUÍ
# Para cada molécula, genera su InChI e InChIKey


# b) Convierte este InChI a SMILES
inchi_misterioso = "InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2/h2-11H,1H2"

# TU CÓDIGO AQUÍ
# ¿Qué molécula es?

### 📝 Ejercicio 2: Crear patrones SMARTS

Crea patrones SMARTS para los siguientes grupos funcionales:

In [ ]:
# Ejercicio 2: Define patrones SMARTS

mis_patrones = {
    "Fenol": "",  # Benceno con -OH
    "Éter": "",  # C-O-C
    "Nitro": "",  # -NO2
    "Halógeno": "",  # F, Cl, Br, I
}

# TU CÓDIGO AQUÍ
# Completa los patrones SMARTS

# Prueba tus patrones con estas moléculas
moleculas_test = {
    "Fenol": "Oc1ccccc1",
    "Anisol": "COc1ccccc1",
    "Nitrobenceno": "O=[N+]([O-])c1ccccc1",
    "Clorobenceno": "Clc1ccccc1"
}

# Verificar tus patrones (código de verificación aquí)

### 📝 Ejercicio 3: Análisis de biblioteca de fármacos

Analiza una biblioteca de fármacos y clasifícalos según sus grupos funcionales:

In [ ]:
# Ejercicio 3: Clasificación de fármacos

farmacos = {
    "Loratadina": "CCOC(=O)N1CCC(=C2c3ccc(Cl)cc3CCc4cccnc42)CC1",
    "Omeprazol": "COc1ccc2nc(S(=O)Cc3ncc(C)c(OC)c3C)[nH]c2c1",
    "Simvastatina": "CCC(C)(C)C(=O)O[C@H]1C[C@@H](C)C=C2C=C[C@H](C)[C@H](CC[C@H]3C[C@H](O)CC(=O)O3)[C@@H]12",
    "Ranitidina": "CNC(=C[N+](=O)[O-])NCCSCC1=CC=C(O1)CN(C)C"
}

# TU CÓDIGO AQUÍ
# 1. Para cada fármaco, identifica qué grupos funcionales contiene
# 2. Calcula propiedades básicas (peso molecular, fórmula)
# 3. Genera una tabla resumen
# 4. Visualiza las estructuras

### 📝 Ejercicio 4: Diseño de filtro molecular

Crea un filtro para encontrar moléculas con características específicas:

In [ ]:
# Ejercicio 4: Filtro molecular avanzado

def filtro_molecular(smiles, criterios):
    """
    Filtra moléculas según criterios específicos.
    
    Args:
        smiles (str): SMILES de la molécula
        criterios (dict): Diccionario con criterios de filtrado
    
    Returns:
        bool: True si cumple todos los criterios
    """
    
    # TU CÓDIGO AQUÍ
    # Implementa el filtro considerando:
    # - Peso molecular (min/max)
    # - Presencia de grupos funcionales
    # - Número de anillos aromáticos
    # - Número de donadores/aceptores de H
    
    pass

# Ejemplo de criterios (Regla de Lipinski)
criterios_lipinski = {
    'peso_molecular_max': 500,
    'logP_max': 5,
    'donadores_H_max': 5,
    'aceptores_H_max': 10
}

# Prueba tu filtro con la biblioteca de fármacos

---

## 📊 Resumen y Conclusiones

### ✅ Lo que hemos aprendido

En esta actividad has dominado:
- **SMILES**: Notación compacta y legible para representar moléculas
- **InChI**: Identificador único estándar internacional
- **SMARTS**: Lenguaje de patrones para búsqueda de subestructuras
- **Conversiones**: Transformación entre diferentes notaciones
- **Aplicaciones prácticas**: Búsqueda y filtrado de compuestos químicos

### 🎯 Puntos Clave

| Notación | Ventajas | Cuándo usar |
|----------|----------|-------------|
| **SMILES** | Compacto, legible | Programación, comunicación |
| **InChI** | Único, estándar | Bases de datos, publicaciones |
| **SMARTS** | Flexible, potente | Búsquedas, filtros |

✅ **Conceptos dominados**:
- Representación textual de estructuras moleculares
- Generación y validación de notaciones químicas
- Búsqueda de subestructuras y grupos funcionales
- Conversión automática entre formatos

✅ **Herramientas utilizadas**:
- RDKit para manipulación de notaciones
- SMILES/InChI para representación molecular
- SMARTS para reconocimiento de patrones

### 🚀 Próximos Pasos
En la siguiente actividad exploraremos **RDKit y OpenBabel** en profundidad: bibliotecas esenciales para manipulación molecular, cálculo de descriptores y análisis estructural.

---

## 📚 Referencias

1. **SMILES Tutorial**: Weininger, D. (1988). *SMILES, a chemical language and information system*. J. Chem. Inf. Comput. Sci. 28, 31-36.
2. **InChI Documentation**: https://www.inchi-trust.org/
3. **RDKit Documentation**: https://www.rdkit.org/docs/
4. **SMARTS Theory**: https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html

---

<div align="center">

## 🎉 ¡Felicitaciones!

Has completado la **Actividad 2.2: Notaciones Químicas**

**Siguiente actividad**: RDKit y OpenBabel

[![Anterior](https://img.shields.io/badge/⬅️_Actividad_2.1-Formatos_Moleculares-blue.svg)](01_formatos_moleculares.ipynb)
[![Siguiente](https://img.shields.io/badge/Actividad_2.3_➡️-RDKit_y_OpenBabel-green.svg)](03_rdkit_openbabel.ipynb)

---

📚 **[Volver al Módulo 2](README.md)** | 🏠 **[Inicio del Curso](../README.md)**

---

**Universidad de Caldas - Departamento de Química**  
*Química Computacional 173G7G*

</div>